In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
# 불필요한 경고가 뜨지 않게..
import warnings
warnings.filterwarnings('ignore')
 
# 데이터 전처리 알고리즘 (비지도 학습)
# 문자열 데이터를 숫자로 변환한다.
from sklearn.preprocessing import LabelEncoder
# 표준편차를 기반으로 표준화 한다.
# 잘못된 학습을 정상화 시키는 목적으로 사용한다.
from sklearn.preprocessing import StandardScaler
# 최소값을 0, 최대값1로 하는 표준화
from sklearn.preprocessing import MinMaxScaler
 
# 데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split
# 교차 검증
from sklearn.model_selection import cross_val_score
 
# 평가 함수
# 정확도 평가 함수
from sklearn.metrics import accuracy_score
# mse 평가 함수
from sklearn.metrics import mean_squared_error
 
# 학습 알고리즘 - 분류
# 최 근접 이웃
# 학습시 : 주어진 데이터를 저장만 한다.
# 예측 : 주변의 데이터를 보고 가장 많은 결과로 결정한다.
from sklearn.neighbors import KNeighborsClassifier
# 선형
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 트리
# 학습시 : 주어진 데이터를 가지고 질문들을 생성한다.
# 예측시 : 질문을 통해 최종 결과를 예측한다.
from sklearn.tree import DecisionTreeClassifier
# 앙상블 - 다수의 알고리즘이 던지는 결과를 취합하여 최종 결과를 결정한다.
# 트리들을 사용한다.
# 학습시 - 데이터를 랜덤하게 섞어서 80%를 추출한다.
# 이렇게 추출된 데이터를 트리의 개수만큼 생성하여 각 트리들에게 주고
# 학습을 수행한다.
# 예측시 - 각 트리가 던지는 결과를 취합하여 다수결의 원칙으로 최종 결과를
# 결정한다.
from sklearn.ensemble import RandomForestClassifier
# 부스팅
# 앙상블 알고리즘이 잘못 예측한 데이터를 다시 학습하는 방식
# 학습과 예측 원리를 앙상블과 동일하다.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
 
# 학습 알고리즘 - 회귀
# 최 근접 이웃
from sklearn.neighbors import KNeighborsRegressor
# 선형
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
# 트리
from sklearn.tree import DecisionTreeRegressor
# 앙상블
from sklearn.ensemble import RandomForestRegressor
# 부스팅
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
 
# 딥러닝
import tensorflow as tf
 
# 딥러닝 모델의 각 층을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형회귀를 수행하는 은닉층
from tensorflow.keras.layers import Dense
# 활성화 함수를 관리하는 것
from tensorflow.keras.layers import Activation
 
# 조기중단
from tensorflow.keras.callbacks import EarlyStopping
# 성능이 개선되면 모델을 파일로 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint
 
# 저장한 딥러닝 모델을 불러온다.
from tensorflow.keras.models import load_model
 
# 원핫 인코딩 하는 함수
from tensorflow.keras.utils import to_categorical
 
import pickle
 
import os
import glob
from PIL import Image


In [2]:
# GPU 사용 설정
# 필요한 만큼만 사용하도록 설정한다.
gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [3]:
# 하위 폴더의 이름을 가져온다.
categories = list(os.walk('image/101_ObjectCategories'))[0][1]
categories


['accordion',
 'airplanes',
 'anchor',
 'ant',
 'barrel',
 'bass',
 'beaver',
 'binocular',
 'bonsai',
 'brain',
 'brontosaurus',
 'buddha',
 'butterfly',
 'camera',
 'cannon',
 'car_side',
 'ceiling_fan',
 'cellphone',
 'chair',
 'chandelier',
 'cougar_body',
 'cougar_face',
 'crab',
 'crayfish',
 'crocodile',
 'crocodile_head',
 'cup',
 'dalmatian',
 'dollar_bill',
 'dolphin',
 'dragonfly',
 'electric_guitar',
 'elephant',
 'emu',
 'euphonium',
 'ewer',
 'Faces',
 'Faces_easy',
 'ferry',
 'flamingo',
 'flamingo_head',
 'garfield',
 'gerenuk',
 'gramophone',
 'grand_piano',
 'hawksbill',
 'headphone',
 'hedgehog',
 'helicopter',
 'ibis',
 'inline_skate',
 'joshua_tree',
 'kangaroo',
 'ketch',
 'lamp',
 'laptop',
 'Leopards',
 'llama',
 'lobster',
 'lotus',
 'mandolin',
 'mayfly',
 'menorah',
 'metronome',
 'minaret',
 'Motorbikes',
 'nautilus',
 'octopus',
 'okapi',
 'pagoda',
 'panda',
 'pigeon',
 'pizza',
 'platypus',
 'pyramid',
 'revolver',
 'rhino',
 'rooster',
 'saxophone',
 'sc

In [4]:
# 이미지 파일들을 가지고 있는 경로 루트
caltech_dir = 'image/101_ObjectCategories'
# 전체 카테고리 개수
nb_class = len(categories)
nb_class


101

In [5]:
# 이미지의 크기
# 이미지를 모두 같은 사이즈로 조정해야 한다.
image_w = 64
image_h = 64
# 전체 픽셀의 개수
# 흑백 : 1, 컬러 : 3
pixels = image_h * image_w * 3

In [6]:
# 이미지  변형작업
# 이미지 데이터들을 담을 리스트
X = []
# 결과 데이터를 담을 리스트
y = []

# 폴더의 수 만큼 반복한다.
for idx, cat in enumerate(categories) :
    # print(idx, cat)
    
    # 원핫 인코딩
    label = []
    # 카테고리의 개수 만큼 0으로 채운 리스트를 생성한다.
    for _ in range(nb_class) :
        label.append(0)
        
    # 현재 폴더의 위치에 1을 설정해준다.
    label[idx] = 1
    
    # 이미지를 읽어올 경로
    image_dir = f'{caltech_dir}/{cat}'
    # print(image_dir)
    
    # 현재 폴더에서 불러올 이미지 파일 목록을 가져온다.
    files = glob.glob(f'{image_dir}/*.jpg')
    
    # 가져온 이미지 파일 수 만큼 반복한다.
    for i, f in enumerate(files) :
        # 이미지를 읽어온다.
        img = Image.open(f)
        # 이미지 파일은 GBR 형태로 되어 있다.
        # CNN을 사용할 것이기 때문에 RGB 형태로 변환한다.
        img = img.convert('RGB')
        # 이미지의 크기를 조정한다.
        img = img.resize((image_h, image_w))
        # 행렬로 변환한다.
        data = np.array(img)
        
        # 리스트에 담는다.
        X.append(data)
        y.append(label)
        
    print(f'{image_dir} 완료')
    
X = np.array(X)
y = np.array(y)

print('작업완료')


image/101_ObjectCategories/accordion 완료
image/101_ObjectCategories/airplanes 완료
image/101_ObjectCategories/anchor 완료
image/101_ObjectCategories/ant 완료
image/101_ObjectCategories/barrel 완료
image/101_ObjectCategories/bass 완료
image/101_ObjectCategories/beaver 완료
image/101_ObjectCategories/binocular 완료
image/101_ObjectCategories/bonsai 완료
image/101_ObjectCategories/brain 완료
image/101_ObjectCategories/brontosaurus 완료
image/101_ObjectCategories/buddha 완료
image/101_ObjectCategories/butterfly 완료
image/101_ObjectCategories/camera 완료
image/101_ObjectCategories/cannon 완료
image/101_ObjectCategories/car_side 완료
image/101_ObjectCategories/ceiling_fan 완료
image/101_ObjectCategories/cellphone 완료
image/101_ObjectCategories/chair 완료
image/101_ObjectCategories/chandelier 완료
image/101_ObjectCategories/cougar_body 완료
image/101_ObjectCategories/cougar_face 완료
image/101_ObjectCategories/crab 완료
image/101_ObjectCategories/crayfish 완료
image/101_ObjectCategories/crocodile 완료
image/101_ObjectCategories/crocodile_

In [7]:
# 학습용과 검증으로 나눈다
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [8]:
# 하나의 듀플에 모두 담는다.
xy = (X_train, X_test, y_train, y_test)
# 저장한다.
np.save('image/Caltech_obj_101.npy', xy)

with open('image/Caltech_classes.dat', 'wb') as fp : 
    pickle.dump(categories, fp)

In [9]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 불필요한 경고가 뜨지 않게..
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘 (비지도 학습)
# 문자열 데이터를 숫자로 변환한다.
from sklearn.preprocessing import LabelEncoder
# 표준편차를 기반으로 표준화 한다.
# 잘못된 학습을 정상화 시키는 목적으로 사용한다.
from sklearn.preprocessing import StandardScaler
# 최소값을 0, 최대값1로 하는 표준화
from sklearn.preprocessing import MinMaxScaler

# 데이터를 학습용과 검증으로 나눈다.
from sklearn.model_selection import train_test_split
# 교차 검증
from sklearn.model_selection import cross_val_score

# 평가 함수
# 정확도 평가 함수
from sklearn.metrics import accuracy_score
# mse 평가 함수
from sklearn.metrics import mean_squared_error

# 학습 알고리즘 - 분류
# 최 근접 이웃
# 학습시 : 주어진 데이터를 저장만 한다.
# 예측 : 주변의 데이터를 보고 가장 많은 결과로 결정한다.
from sklearn.neighbors import KNeighborsClassifier
# 선형
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 트리
# 학습시 : 주어진 데이터를 가지고 질문들을 생성한다.
# 예측시 : 질문을 통해 최종 결과를 예측한다.
from sklearn.tree import DecisionTreeClassifier
# 앙상블 - 다수의 알고리즘이 던지는 결과를 취합하여 최종 결과를 결정한다.
# 트리들을 사용한다.
# 학습시 - 데이터를 랜덤하게 섞어서 80%를 추출한다.
# 이렇게 추출된 데이터를 트리의 개수만큼 생성하여 각 트리들에게 주고
# 학습을 수행한다.
# 예측시 - 각 트리가 던지는 결과를 취합하여 다수결의 원칙으로 최종 결과를
# 결정한다.
from sklearn.ensemble import RandomForestClassifier
# 부스팅
# 앙상블 알고리즘이 잘못 예측한 데이터를 다시 학습하는 방식
# 학습과 예측 원리를 앙상블과 동일하다.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# 학습 알고리즘 - 회귀
# 최 근접 이웃
from sklearn.neighbors import KNeighborsRegressor
# 선형
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
# 트리
from sklearn.tree import DecisionTreeRegressor
# 앙상블
from sklearn.ensemble import RandomForestRegressor
# 부스팅
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

# 딥러닝
import tensorflow as tf

# 딥러닝 모델의 각 층을 관리하는 객체
from tensorflow.keras.models import Sequential
# 선형회귀를 수행하는 은닉층
from tensorflow.keras.layers import Dense
# 활성화 함수를 관리하는 것
from tensorflow.keras.layers import Activation

# 조기중단
from tensorflow.keras.callbacks import EarlyStopping
# 성능이 개선되면 모델을 파일로 자동 저장한다.
from tensorflow.keras.callbacks import ModelCheckpoint

# 저장한 딥러닝 모델을 불러온다.
from tensorflow.keras.models import load_model

# 원핫 인코딩 하는 함수
from tensorflow.keras.utils import to_categorical

import pickle

import os
import glob
from PIL import Image
